In [1]:
from rich import print

from cali._plate_viewer._analysis_new import AnalysisRunner
from cali.readers import TensorstoreZarrReader
from cali.sqlmodel import print_experiment_tree, save_experiment_to_db

# Run from Database Example

In [2]:
from cali.sqlmodel import load_experiment_from_db

In [4]:
database_path = (
    "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis/cali.db"  # noqa: E501
)
exp = load_experiment_from_db(database_path)
print_experiment_tree(exp)

🧪 evoked
├── Experiment Type: Evoked Activity
├── Imported from tests/test_data/evoked/evk_analysis
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: c1, t1
        └── 📷 B5_0000_p0 (fov: 0 - pos: 0)
            ├── 🔬 ROI 1 - 🔋 active - ✨ spontaneous
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            ├── 🔬 ROI 2 - 🔋 active - ⚡️ stimulated
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            ├── 🔬 ROI 3 - 🔋 active - ⚡️ stimulated
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            └── 🔬 ROI 4 - 🔋 active - ✨ spontaneous
                ├── 📊 Trace data available
                ├── 📈 Data analysis available
                └── 🎭 ROI mask available

In [4]:
# delete existing rois
for well in exp.plate.wells:
    for fov in well.fovs:
        fov.rois = []
# save database after deletion
save_experiment_to_db(exp, database_path, overwrite=True)

print_experiment_tree(exp)

🧪 evoked
├── Experiment Type: Evoked Activity
├── Imported from /Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: c1, t1
        └── 📷 B5_0000_p0 (fov: 0 - pos: 0)

In [5]:
runner = AnalysisRunner()
runner.set_experiment(exp)
data = TensorstoreZarrReader(exp.data_path)
# print("data:", data.store)
runner.set_data(data)
# print(runner.get_settings())


def _p(msg: str) -> None:
    print("ANALYSIS INFO:", msg)


runner.analysisInfo.connect(_p)

<function __main__._p(msg: str) -> None>

In [6]:
runner.run()

2025-11-09 14:03:41,693 - cali_logger - INFO - Starting Analysis...


2025-11-09 14:03:41,694 - cali_logger - INFO - Starting traces analysis...
2025-11-09 14:03:41,697 - cali_logger - INFO - Threads: 1
2025-11-09 14:03:41,725 - cali_logger - INFO - Extracting Traces Data from Well B5_0000_p0.
Extracting Traces Data from Well B5_0000_p0.:   0%|          | 0/4 [00:00<?, ?it/s]/Users/fdrgsp/Documents/git/cali/.venv/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 153, using nperseg = 153
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/Users/fdrgsp/Documents/git/cali/.venv/lib/python3.10/site-packages/oasis/functions.py:817: FutureWarning: Beginning in SciPy 1.17, multidimensional input will be treated as a batch, not `ravel`ed. To preserve the existing behavior and silence this warning, `ravel` arguments before passing them to `toeplitz`.
  A = scipy.linalg.toeplitz(xc[np.arrange(lags)],
Extracting Traces Data from Well B5_0000_p0.: 100%|██████████| 4/4 [00:00

ANALYSIS INFO: Position 0 completed.

2025-11-09 14:03:41,758 - cali_logger - INFO - Analysis Finished.
2025-11-09 14:03:41,758 - cali_logger - INFO - Saving results to database...
2025-11-09 14:03:41,795 - cali_logger - INFO - Results saved to database '/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis/cali.db'.


In [7]:
print_experiment_tree(exp)

🧪 evoked
├── Experiment Type: Evoked Activity
├── Imported from /Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: c1, t1
        └── 📷 B5_0000_p0 (fov: 0 - pos: 0)
            ├── 🔬 ROI 1 - 🔋 active - ✨ spontaneous
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            ├── 🔬 ROI 2 - 🔋 active - ⚡️ stimulated
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            ├── 🔬 ROI 3 - 🔋 active - ⚡️ stimulated
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            └── 🔬 ROI 4 - 🔋 active - ✨ spontaneous
                ├── 📊 Trace data available
                ├── 📈 Data analysis available
                └── 🎭 ROI mask available

# Run From Directories Example

In [ ]:
from datetime import datetime

import useq

from cali.sqlmodel import AnalysisSettings, Experiment, useq_plate_plan_to_db

In [27]:
database_path = "/Users/fdrgsp/Desktop/cali_test/cali_new.db"
data_path = (
    "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk.tensorstore.zarr"  # noqa: E501
)
analysis_path = "/Users/fdrgsp/Desktop/cali_test"
labels_path = "/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_labels"

new_exp = Experiment(
    id=0,
    name="New Experiment",
    description="A Test Experiment.",
    created_at=datetime.now(),
    database_path=database_path,
    data_path=data_path,
    labels_path=labels_path,
    analysis_path=analysis_path,
)
print(new_exp)

Experiment(
    id=0,
    name='New Experiment',
    description='A Test Experiment.',
    created_at=datetime.datetime(2025, 11, 9, 14, 10, 3, 880162),
    database_path='/Users/fdrgsp/Desktop/cali_test/cali_new.db',
    data_path='/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk.tensorstore.zarr',
    labels_path='/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_labels',
    analysis_path='/Users/fdrgsp/Desktop/cali_test',
    experiment_type='Spontaneous Activity',
    positions_analyzed=[]
)

In [28]:
# load the data and get the useq plate plan from the sequence
data = TensorstoreZarrReader(new_exp.data_path)
plate_plan = data.sequence.stage_positions
assert isinstance(plate_plan, useq.WellPlatePlan)

# Define plate maps for conditions
plate_maps = {
    "genotype": {"B5": "WT"},
    "treatment": {"B5": "Vehicle"},
}

# Create plate with plate_maps and conditions in one step
plate = useq_plate_plan_to_db(plate_plan, new_exp, plate_maps)
new_exp.plate = plate

print(new_exp.plate)
print_experiment_tree(new_exp)

Plate(
    name='96-well',
    plate_type='96-well',
    rows=8,
    columns=12,
    plate_maps={'genotype': {'B5': 'WT'}, 'treatment': {'B5': 'Vehicle'}},
    experiment_id=0,
    id=None
)

🧪 New Experiment
├── Experiment Type: Spontaneous Activity
├── A Test Experiment.
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: WT, Vehicle

In [29]:
analysis_settings = AnalysisSettings(
    neuropil_inner_radius=0,
    peaks_prominence_multiplier=1.0,
    spikes_sync_cross_corr_lag=5,
    neuropil_min_pixels=0,
    calcium_sync_jitter_window=5,
    led_power_equation="y = 11 * x - 6",
    neuropil_correction_factor=0.0,
    calcium_network_threshold=90.0,
    led_pulse_duration=100,
    led_pulse_powers=[2.0, 4.0, 6.0],
    led_pulse_on_frames=[3, 53, 103],
    decay_constant=0.0,
    spike_threshold_value=1.0,
    stimulation_mask_path="/Users/fdrgsp/Documents/git/cali/tests/test_data/evoked/evk_analysis/stimulation_mask.tif",
    id=1,
    dff_window=30,
    spike_threshold_mode="multiplier",
    threads=1,
    peaks_height_value=3.0,
    burst_threshold=30.0,
    experiment_id=1,
    peaks_height_mode="multiplier",
    burst_min_duration=3,
    stimulation_mask_id=1,
    created_at=datetime(2025, 11, 8, 20, 25, 1, 911683),
    peaks_distance=2,
    burst_gaussian_sigma=2.0,
)
new_exp.analysis_settings = analysis_settings

print_experiment_tree(new_exp)

🧪 New Experiment
├── Experiment Type: Spontaneous Activity
├── A Test Experiment.
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: WT, Vehicle

In [40]:
runner = AnalysisRunner()
runner.set_experiment(new_exp)
data = TensorstoreZarrReader(new_exp.data_path)
runner.set_data(data)

# set which positions to analyze
new_exp.positions_analyzed = list(range(len(plate_plan)))
print(f"Positions to analyze: {new_exp.positions_analyzed}")


def _p(msg: str) -> None:
    print("ANALYSIS INFO:", msg)


runner.analysisInfo.connect(_p)

Positions to analyze: [0]

<function __main__._p(msg: str) -> None>

In [41]:
runner.run()

2025-11-09 14:14:06,948 - cali_logger - INFO - Starting Analysis...
2025-11-09 14:14:06,949 - cali_logger - INFO - Starting traces analysis...
2025-11-09 14:14:06,950 - cali_logger - INFO - Number of threads for analysis: 1
2025-11-09 14:14:06,989 - cali_logger - INFO - Extracting Traces Data from Well B5_0000_p0.


Positions to analyze: [0]


Extracting Traces Data from Well B5_0000_p0.:   0%|          | 0/4 [00:00<?, ?it/s]/Users/fdrgsp/Documents/git/cali/.venv/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:790: UserWarning: nperseg = 256 is greater than input length  = 153, using nperseg = 153
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,
/Users/fdrgsp/Documents/git/cali/.venv/lib/python3.10/site-packages/oasis/functions.py:817: FutureWarning: Beginning in SciPy 1.17, multidimensional input will be treated as a batch, not `ravel`ed. To preserve the existing behavior and silence this warning, `ravel` arguments before passing them to `toeplitz`.
  A = scipy.linalg.toeplitz(xc[np.arrange(lags)],
Extracting Traces Data from Well B5_0000_p0.: 100%|██████████| 4/4 [00:00<00:00, 106.39it/s]
2025-11-09 14:14:07,035 - cali_logger - INFO - Position 0 completed.


ANALYSIS INFO: Position 0 completed.

2025-11-09 14:14:07,036 - cali_logger - INFO - Analysis Finished.
2025-11-09 14:14:07,036 - cali_logger - INFO - Saving results to database...
2025-11-09 14:14:07,067 - cali_logger - INFO - Results saved to database '/Users/fdrgsp/Desktop/cali_test/cali_new.db'.


In [42]:
exp_after_analysis = load_experiment_from_db(database_path)
print_experiment_tree(exp_after_analysis)

🧪 New Experiment
├── Experiment Type: Spontaneous Activity
├── A Test Experiment.
├── ⚙️ Analysis Settings available
└── 📋 96-well (96-well)
    └── 🧫 B5 - 🧪 Conditions: WT, Vehicle
        └── 📷 B5_0000_p0 (fov: 0 - pos: 0)
            ├── 🔬 ROI 1 - 🔋 active - ✨ spontaneous
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            ├── 🔬 ROI 2 - 🔋 active - ✨ spontaneous
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            ├── 🔬 ROI 3 - 🔋 active - ✨ spontaneous
            │   ├── 📊 Trace data available
            │   ├── 📈 Data analysis available
            │   └── 🎭 ROI mask available
            └── 🔬 ROI 4 - 🔋 active - ✨ spontaneous
                ├── 📊 Trace data available
                ├── 📈 Data analysis available
                └── 🎭 ROI mask available